In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
# def scraping_utmb():
#     url = 'https://utmb.world/utmb-index/runner-search'
#     browsers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \(KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36'}
#     page = requests.get(url, headers=browsers)

#     soup = BeautifulSoup(page.content, 'html.parser')
#     runners = soup.select('.my-table_row__nlm_j')

#     dict_names = {
#         i.find('a', attrs={'class': 'link_link__96ppl link-underline'}).text: [i.find('div', attrs={'class': 'my-table_cell__z__zN col-2'}).text,
#         str(i.find('div', attrs={'data-label': "nationality"}))[180:182],
#         i.find('div', attrs={'data-label': "Gender"}).text,
#         i.find('div', attrs={'data-label': "Age Group"}).text, 
#         'https://utmb.world/'+ str(i.find('a', attrs={'class': 'link_link__96ppl link-underline'})['href'])
#         ]
#         for i in runners
#     }

#     return dict_names

# resultado = scraping_utmb()

# df = pd.DataFrame(resultado.values(), index=resultado.keys())
# df = df.set_axis(['Posição', 'Nacionalidade', 'Gênero', 'Faixa Etária', 'Site'], axis=1)
# df


---
Selenium
---

In [3]:

# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager 
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from time import sleep

# Funções

In [4]:
# def extrair_dados_pagina(driver):
#     runners = driver.find_elements(by=By.CSS_SELECTOR, value=".my-table_row__nlm_j")
    
#     dict_names = {}
#     for runner in runners:
#         name = runner.find_element(by=By.XPATH, value="//a[@class='link_link__96ppllink-underline']").text
#         nationality = runner.find_element(by=By.XPATH, value="//div[@class='index-ranking-table_cell_nationality__Pskf9 my-table_cell__z__zN col-2 my-table_hide_mobile__t6aJD']").text
#         gender = runner.find_element(by=By.XPATH, value="//a[@class='index-ranking-table_cell_gender__svRts my-table_cell__z__zN col-2 my-table_hide_mobile__t6aJD']").text
#         age_group = runner.find_element(by=By.XPATH, value="//a[@class='index-ranking-table_cell_age_group__WOVLq my-table_cell__z__zN col-2 my-table_hide_mobile__t6aJD']").text
#         link = runner.find_element(by=By.XPATH, value="//a[@href='.link_link__96ppl.link-underline]").text
        
#         dict_names[name] = [nationality, gender, age_group, link]

#     return dict_names


In [5]:
# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service)

# driver.get(url='https://utmb.world/utmb-index/runner-search')

# accept = driver.find_element(by=By.ID, value="onetrust-accept-btn-handler")
# accept.click()

# parameters = driver.find_elements(by=By.XPATH, value="//div[@class='index-ranking-table_cell_nationality__Pskf9 my-table_cell__z__zN col-2 my-table_hide_mobile__t6aJD']")
# parameter = parameters[10]
# nacionalidade_button = driver.find_element(by=By.XPATH, value="//button[@aria-controls='menu--7']")
# driver.execute_script("arguments[0].scrollIntoView();", parameter)
# wait = WebDriverWait(driver, 10)
# wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@class='index-ranking-table_cell_nationality__Pskf9 my-table_cell__z__zN col-2 my-table_hide_mobile__t6aJD']")))
# nacionalidade_button.click() 

# brazil_option = driver.find_element(by=By.XPATH, value="//div[@id='option-27--menu--7']")
# brazil_option.click()

# current_page_data = extrair_dados_pagina(driver)

# #next = driver.find_elements(by=By.TAG_NAME, value="a")
# #proximo = next[98].get_attribute('aria-label')
# #proximo.click()"

# Extraindo dados pelo JSON da API

In [6]:
with open('runnersbrasileiros.json', 'r') as f:
    data = json.load(f)

runners_list = data['runners']

new_data = [{'id': runner['id'], 
             'fullname': runner['fullname'], 
             'ageGroup': runner['ageGroup'],
             'uri': runner['uri'],
             'sex': runner['sex'],
             'ip': runner['ip']} 
            for runner in runners_list]

df = pd.DataFrame(new_data)
df


,id,fullname,ageGroup,uri,sex,ip
0,1359763,Rogerio SILVESTRIN,35-39,1359763.rogerio.silvestrin,H,852
1,2623403,Wellington ROBERTO NORONHA,35-39,2623403.wellington.robertonoronha,H,830
2,395551,Celio AUGUSTO ROSA,45-49,395551.celio.augustorosa,H,824
3,5027328,Johnny LUNA LIMA,20-34,5027328.johnny.lunalima,H,814
4,5268870,Cordeiro Valdenir JANDOSA,45-49,5268870.cordeirovaldenir.jandosa,H,807
...,...,...,...,...,...,...
36430,5878517,Fabiane SILVA,20-34,5878517.fabiane.silva,F,0
36431,5878579,ROGERIO SOUZA,45-49,5878579.rogerio.souza,H,0
36432,5878599,Gleidson ANDRADE,20-34,5878599.gleidson.andrade,H,0
36433,5878635,Luciano Antônio RIBEIRO,35-39,5878635.lucianoantonio.ribeiro,H,0


# Extraindo dados direto da API

In [7]:
url_utmb_brazil = 'https://api.utmb.world/search/runners?category=general&sex=&ageGroup=&nationality=BR&limit=1000000&offset=0&search='

runners = requests.get(url_utmb_brazil)
runners_br_df = pd.DataFrame(runners.json() ["runners"])
runners_br_df

,id,ageGroup,fullname,uri,ip,nationality,sex,picture
0,1359763,35-39,Rogerio SILVESTRIN,1359763.rogerio.silvestrin,855,BR,H,None
1,2623403,35-39,Wellington ROBERTO NORONHA,2623403.wellington.robertonoronha,828,BR,H,worldseries/Members/c57878f6-56e2-4697-a961-35...
2,395551,45-49,Celio AUGUSTO ROSA,395551.celio.augustorosa,826,BR,H,None
3,5027328,20-34,Johnny LUNA-LIMA,5027328.johnny.lunalima,814,BR,H,None
4,5268870,45-49,Cordeiro Valdenir JANDOSA,5268870.cordeirovaldenir.jandosa,807,BR,H,None
...,...,...,...,...,...,...,...,...
36665,5886925,20-34,Tarik Baracat Ajub,5886925.tarik.baracatajub,0,BR,H,None
36666,5887159,40-44,José Fagner SOARES FÉLIX,5887159.josefagner.soaresfelix,0,BR,H,None
36667,5888579,35-39,Rodrigo DE OLIVEIRA SILVA,5888579.rodrigo.deoliveirasilva,0,BR,H,None
36668,5889010,20-34,Bernardo OLIVEIRA FIUZA,5889010.bernardo.oliveirafiuza,0,BR,H,worldseries/Members/acdc5aa5-88fb-420c-9ec5-67...


In [8]:
from corredores import corredor_corridas, corredor_info

corredor_corridas(runners_br_df['uri'][0])



,date,dateIso,race,piCategory,distance,elevationGain,time,hasCotation,isCotationPending,isDnf,utmbEventStatus,rank,rankGender,totalRanked,totalRankedGender,index,uri,country,countryCode
0,02.04.2024,2024-02-04,Cambotas Trail Fest 2024 - Cambotas Marathon,50k,44.70,2497,04:29:02,True,False,False,,1,1,122,98,None,31151.cambotastrailfestcambotasmarathon.2024,Brazil,BR
1,11.11.2023,2023-11-11,Mons Ultra Trail 2023 - Mons 100k,100k,104.50,6100,14:00:51,True,False,False,,2,2,32,24,None,23757.monsultratrailmons100k.2023,Brazil,BR
2,10.21.2023,2023-10-21,Indomit Bombinhas 2023 - Team,50k,43.40,1490,04:06:24,True,False,False,,1,1,134,73,None,29831.indomitbombinhasteam.2023,Brazil,BR
3,10.21.2023,2023-10-21,Indomit Bombinhas 2023 - Solo,50k,43.40,1490,03:50:47,True,False,False,,1,1,124,81,None,29830.indomitbombinhassolo.2023,Brazil,BR
4,09.22.2023,2023-09-22,Paraty By UTMB 2023 - Ultra-Trail Serra da Boc...,100k,107.00,4829,11:54:38,True,False,False,event,1,1,334,280,None,35706.paratybyutmbultra-trailserradabocaina130...,Brazil,BR
5,08.19.2023,2023-08-19,La Mision Brasil 2023 - La Mision Brasil 80 Km,100k,81.00,5300,11:35:49,True,False,False,,2,2,199,165,None,11267.lamisionbrasillamisionbrasil80km.2023,Brazil,BR
6,07.16.2023,2023-07-16,Ultramaratona dos Perdidos 2023 - 50k,50k,51.00,3030,05:34:40,True,False,False,,1,1,138,110,None,2419.ultramaratonadosperdidos50k.2023,Brazil,BR
7,04.22.2023,2023-04-22,Jaragua SkyMarathon 2023 - Sky Ultra,50k,49.00,3000,04:57:41,True,False,False,,1,1,101,78,None,33140.jaraguaskymarathonskyultra.2023,Brazil,BR
8,03.12.2023,2023-03-12,Indomit Pedra do Baú 2023 - Indomit Baú 50k,50k,50.80,3280,05:00:00,True,False,False,,1,1,210,170,None,3520.indomitpedradobauindomitbau50k.2023,Brazil,BR
9,12.11.2022,2022-12-11,Cambotas Trail Fest 2022 - Cambotas Marathon,50k,45.60,2500,04:41:30,True,False,False,,1,1,72,59,None,31151.cambotastrailfestcambotasmarathon.2022,Brazil,BR
